In [41]:
# Imports

import pandas as pd
import json
import sklearn
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV, StratifiedShuffleSplit
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA, PCA, SparsePCA
from sklearn.feature_selection import mutual_info_classif, f_classif, SelectKBest, chi2, SelectPercentile
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures, PowerTransformer
from sklearn.pipeline import Pipeline


In [52]:
def tfidf_tokenizer(self):
  return self

In [53]:
with open("/content/domain1_train_data.json") as f:
    df_domain1 = pd.read_json(f, lines=True)

# TF-IDF
TFIDF_MAX_FEATURES = 3000
tfidf_vect = TfidfVectorizer(max_df=0.5,
                             max_features = TFIDF_MAX_FEATURES,
                             lowercase=False, tokenizer=tfidf_tokenizer)
X_new = tfidf_vect.fit_transform(df_domain1['text'])
y = df_domain1['label']


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [43]:
# SVM grid search for params
C_range = np.logspace(1, 3, 3)
gamma_range = np.logspace(-1, 2, 4)
param_grid = dict(kernel=['linear', 'rbf'], C=C_range, gamma=gamma_range)

grid = GridSearchCV(SVC(), param_grid=param_grid, refit = True, verbose = 3)
grid.fit(X_new, y)

print("The best parameters are %s with a score of %0.2f" % (grid.best_params_, grid.best_score_))


Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV 1/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.726 total time=  10.4s
[CV 2/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.674 total time=  10.3s
[CV 3/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.668 total time=   9.9s
[CV 4/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.686 total time=   9.8s
[CV 5/5] END ..C=10.0, gamma=0.1, kernel=linear;, score=0.688 total time=  10.0s
[CV 1/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.732 total time=  11.7s
[CV 2/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.733 total time=  11.6s
[CV 3/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.701 total time=  11.8s
[CV 4/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.721 total time=  11.9s
[CV 5/5] END .....C=10.0, gamma=0.1, kernel=rbf;, score=0.714 total time=  12.0s
[CV 1/5] END ..C=10.0, gamma=1.0, kernel=linear;, score=0.726 total time=  10.4s
[CV 2/5] END ..C=10.0, gamma=1.0, kernel=linear

In [44]:
# The best parameters are {'C': 100.0, 'gamma': 1.0, 'kernel': 'rbf'} with a cv score of 0.78
svm_domain1 = SVC(C=100.00, gamma=1.0, kernel='rbf')
svm_domain1.fit(X_new, y)

SVC(C=100.0, gamma=1.0)

In [54]:
# Export model
import pickle as pkl
with open("svm_domain1.pkl", "wb") as writefile:
    pkl.dump([tfidf_vect, svm_domain1], writefile)

In [57]:
# Export pipeline
pipe = Pipeline([
    ('tfidf', tfidf_vect),
    ('svc', svm_domain1)
])

import pickle as pkl
with open("joey_svm_domain1_pipeline.pkl", "wb") as writefile:
    pkl.dump([pipe, tfidf_vect, tfidf_tokenizer, svm_domain1], writefile)


In [58]:
# Test data prediction
with open("/content/test_data.json") as f:
    df_test = pd.read_json(f, lines=True)

pred_svm_domain1 = pipe.predict(df_test['text'])


In [59]:

# Save Predictions to CSV
output_df = pd.DataFrame({'id': df_test['id'], 'class': pred_svm_domain1})
output_csv_path = '/content/test_predictions.csv'
output_df.to_csv(output_csv_path, index=False)